In [1]:
from os import listdir
import numpy as np

from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img


Using TensorFlow backend.


In [2]:
train_dir = "eval_light/"

In [3]:
def load_text(filename):
    
    file = open(filename, "r")
    text = file.read()
    file.close()
    
    return text

def load_data(data):
    
    imgs = []
    texts = []
    
    for file in listdir(data):
        
        if file[-3:] == "npz":
            
            image = np.load(data + file)
            imgs.append(image['features'])
        
        else:
            
            syntax = "<START> " + load_text(data + file) + " <END>"
            syntax = ' '.join(syntax.split())
            syntax = syntax.replace(',', ' ,')
            
            texts.append(syntax)
            
    #images = np.array(imgs, dtype = float)
    return imgs, texts 


In [4]:
images, texts = load_data(train_dir)

In [5]:
#preprocess the data

tokenizer = Tokenizer(filters='', split=" ", lower=False)
tokenizer.fit_on_texts([load_text('bootstrap.vocab')])

vocab_size = len(tokenizer.word_index) + 1 #for the empty space (?)

train_seq = tokenizer.texts_to_sequences(texts)
max_seq = max(len(s) for s in train_seq)
max_length = 48

In [6]:
def preprocess_data(sequences, features):
    X, y, image_data = list(), list(), list()
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_seq)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-48:])
            y.append(out_seq)
    return np.array(X), np.array(y), np.array(image_data)

X, y, image_data = preprocess_data(train_seq, images)

In [7]:
#Create the encoder
image_model = Sequential()
image_model.add(Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3,)))
image_model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(max_length))

visual_input = Input(shape=(256, 256, 3,))
encoded_image = image_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = LSTM(128, return_sequences=True)(language_model)

#Create the decoder
decoder = concatenate([encoded_image, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[visual_input, language_input], outputs=decoder)
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
#Save the model for every 2nd epoch
filepath="org-weights-epoch-{epoch:04d}--val_loss-{val_loss:.4f}--loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=2)
callbacks_list = [checkpoint]

In [9]:
model.fit([image_data, X], y, batch_size=1, shuffle=False, validation_split=0.1, callbacks=callbacks_list, verbose=1, epochs=50)

Train on 624 samples, validate on 70 samples
Epoch 1/50
624/624 [==============================] - 1119s 2s/step - loss: 2.5490 - val_loss: 2.4863
Epoch 2/50
624/624 [==============================] - 1105s 2s/step - loss: 2.4617 - val_loss: 2.4617

Epoch 00002: saving model to org-weights-epoch-0002--val_loss-2.4617--loss-2.4617.hdf5
Epoch 3/50
624/624 [==============================] - 1111s 2s/step - loss: 2.4341 - val_loss: 2.3690
Epoch 4/50
624/624 [==============================] - 353s 566ms/step - loss: 2.3971 - val_loss: 2.3453

Epoch 00004: saving model to org-weights-epoch-0004--val_loss-2.3453--loss-2.3971.hdf5
Epoch 5/50
624/624 [==============================] - 263s 422ms/step - loss: 2.3608 - val_loss: 2.3388
Epoch 6/50
624/624 [==============================] - 264s 423ms/step - loss: 2.3536 - val_loss: 2.2735

Epoch 00006: saving model to org-weights-epoch-0006--val_loss-2.2735--loss-2.3536.hdf5
Epoch 7/50
624/624 [==============================] - 284s 456ms/step - l

KeyboardInterrupt: 

In [10]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']